In [153]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Rescaling, MaxPooling2D, Activation
from tensorflow.keras.utils import to_categorical, get_file, to_categorical
from PIL import Image
import os
import pandas as pd
import numpy as np
import random

---

# **Data**

In [154]:
import pathlib

data_dir = pathlib.Path('data/')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)


2


In [155]:
data_dir = pathlib.Path('data/')
image_paths = list(data_dir.glob('*/*.jpg'))

data = []
for image_path in image_paths:
    label = image_path.parent.name
    data.append((str(image_path), label))

df = pd.DataFrame(data, columns=['image', 'label'])

In [156]:
img_size = 125

def load_image(image_path):
    img = Image.open(image_path)
    img = img.resize((img_size, img_size))
    return np.array(img)

df['image'] = df['image'].apply(load_image)
df

,image,label
0,"[[[123, 137, 144, 255], [157, 162, 159, 255], ...",Chène
1,"[[[74, 124, 175], [74, 124, 174], [74, 124, 17...",Caroubier


In [157]:
(df['image'].iloc[0]).shape


(125, 125, 4)

In [ ]:
cat = ['Caroubier', 'Chène', 'Faux-poivrier', 'Frène', 'Pistachier', 'Tipa']
df['label'] = pd.Categorical(df['label'])
df['label'] = df['label'].cat.codes
df['label'] = to_categorical(df['label'])

In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [159]:
df

,image,label
0,"[[[123, 137, 144, 255], [157, 162, 159, 255], ...",Chène
1,"[[[74, 124, 175], [74, 124, 174], [74, 124, 17...",Caroubier


---

# **Model**

In [160]:
model = Sequential()

model.add(Conv2D(32, 3, input_shape = (img_size, img_size, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, 3))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, 3))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, 3))
model.add(Activation("relu"))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acuuracy'])

/opt/homebrew/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_29 (Conv2D)              │ (None, 123, 123, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_20 (Activation)      │ (None, 123, 123, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_21 (Activation)      │ (None, 59, 59, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 27, 27, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_22 (Activation)      │ (None, 27, 27, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 13, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 11, 11, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_23 (Activation)      │ (None, 11, 11, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 7744)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       991,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,085,124 (4.14 MB)

 Trainable params: 1,085,124 (4.14 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(df['image'], df['label'])